In [ ]:
%cd /home/tuyendv/Desktop/reranker/ 
from utils import load_data
from sklearn.model_selection import train_test_split
import random
import json

In [ ]:
path = "data/ir/train"
data = load_data(path)
data = list(data)

In [ ]:
random.shuffle(data)
train, valid = train_test_split(data, test_size=0.05, random_state=42)

In [ ]:
len(train)

In [ ]:
len(valid)

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")

data_size = 50000
n = int(len(train)/data_size)

for i in range(n):
    path = f"data/qa_data/train/{str(i).zfill(6)}.json"
    save_data(path, data=train[i*data_size:(i+1)*data_size])

In [ ]:
save_data("data/qa_data/val/000000.json", data=valid)

### statis

In [ ]:
import json

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/raw/squad-train-v2.0-en.json"
data = json.load(open(path))

In [ ]:
msmacro_data = []
for text in data["data"]:
    for paragraph in text["paragraphs"]:
        passage = paragraph["context"]
        for sample in paragraph["qas"]:
            query = sample["question"]
            
            _temp = {
                "query": query,
                "passage": passage
            }
            msmacro_data.append(_temp)

In [ ]:
save_data("/home/tuyendv/Desktop/reranker/data/ir/train/squad-train-v2.0-en.json", msmacro_data)

### vi_squad

In [ ]:
%cd /home/tuyendv/Desktop/reranker
import json
import pandas as pd
from bm25 import BM25
from tqdm import tqdm
from utils import save_data, load_file

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/raw/squad-train-v2.0-en.json"
data = load_file(path)

df = pd.DataFrame(data, columns=["passage", "query", "answer"])
df.head()

In [ ]:
passages = []
for i, doc in enumerate(df.passage.unique().tolist()):
    _id = str(i).zfill(6)
    _text = doc
    
    passages.append([_id, _text])
    
passages = pd.DataFrame(passages, columns=["id", "passage_text"])
passages.head()

In [ ]:
bm25 = BM25()
bm25.train(passages)

In [ ]:
data = []
for index in tqdm(df.index):
    query = df["query"][index]
    positive_passage = df["passage"][index]
    all_passage = bm25.ranking(positive_passage, top_k=8)["retrieval_text"].tolist()
    
    passages = []
    mark = 0
    for passage in all_passage:
        if positive_passage == passage:
            sample = {
                "is_selected":1,
                "passage_text":passage
            }
            mark = 1
        else:
            sample = {
                "is_selected":0,
                "passage_text":passage
            }
        passages.append(sample)
        
    if mark == 0:
        sample = {
            "is_selected":1,
            "passage_text":positive_passage
        }
        passages.append(sample)
    
    data.append(
        {
            "query":query,
            "passages":passages
        }
    )

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/ir/train/squad-train-v2.0-en.json"
save_data(path, data=data)

### vi_data

In [ ]:
import pandas as pd
import json

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/raw/vi_data.json"
df = pd.read_json(path)
df.head()

In [ ]:
data = []
for index in df.index:
    query = df["question"][index]
    positive = df["positive_sample"][index]
    negative_samples = df["negative_samples"][index]
    
    passages = []
    
    passages.append(
        {
            "passage_text":positive,
            "is_selected":1
        }
    )
    
    for negative in negative_samples:
        passages.append(
            {
                "passage_text":negative,
                "is_selected":0
            }
        )
        
    data.append(
        {
            "query":query,
            "passages":passages
        }
    )

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")

path="/home/tuyendv/Desktop/reranker/data/ir/vi_data.json"
save_data(path, data)

### v1

In [ ]:
import json

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/ir/train/ms-macro-train_v1.1.json"

data = []
with open(path, "r", encoding="utf-8") as f:
    for line in f.readlines():
        line = json.loads(line.strip())
        passages = [
            {
                "passage_text":i["passage_text"],
                "is_selected":i["is_selected"]
                } for i in line["passages"]
            ]
        data.append(
            {
                "query":line["query"],
                "passages":passages
            }
        )

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")

path = "/home/tuyendv/Desktop/reranker/data/ir/train/ms-macro-train_v1.1.json"
save_data(path, data[0:30000])

### process data

In [ ]:
%cd /home/tuyendv/Desktop/reranker/
import json
import pandas as pd
from utils import load_file, load_data

In [ ]:
path = '/home/tuyendv/Desktop/reranker/data/processed-ir-data/train'
data = load_data(path)

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
def get_length(query, passages):
    query_length = len(query.split())
    
    lengths = []
    for pa in passages:
        tmp = pa["passage_text"]
        
        lengths.append(len(tmp.split()) + query_length)
        
    return lengths

df["length"] = df.apply(lambda x: get_length(x["query"], x["passages"]), axis=1)

In [ ]:
df["num_pass"] = df.passages.apply(lambda x: len(x))
df.head()

In [ ]:
lengths = [i for j in df.length.tolist() for i in j]
pd.DataFrame(lengths, columns=["lengths"]).lengths.hist(bins=100)

### temp

In [ ]:
%cd /home/tuyendv/Desktop/reranker/
import os 
from glob import glob
import json
import pandas as pd
from utils import load_data

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/ir/test"
data = load_data(path)
data = [json.loads(line) for line in data]
len(data)

In [ ]:
data= pd.DataFrame(data)
data.head()

In [ ]:
def get_length(query, passages):
    query_length = len(query.split())
    
    lengths = []
    for pa in passages:
        tmp = pa["passage_text"]
        
        lengths.append(len(tmp.split()) + query_length)
        
    return lengths

data["length"] = data.apply(lambda x: get_length(x["query"], x["passages"]), axis=1)

In [ ]:
lengths = [i for j in data.length.tolist() for i in j]
pd.DataFrame(lengths, columns=["lengths"]).lengths.hist(bins=100)

In [ ]:
lengths = [i for j in data.length.tolist() for i in j]
pd.DataFrame(lengths, columns=["lengths"]).lengths.hist(bins=100)